In [1]:
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, product, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union, Iterable
from dataclasses import dataclass


import operator
import math
import ast
import sys
import re

In [2]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> Dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

In [3]:
def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    """Return first item in iterable, or default."""
    return next(iter(iterable), default)

def rest(sequence) -> object: 
    return sequence[1:]

def multimap(items: Iterable[Tuple]) -> dict:
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def ints(text: str) -> Tuple[int]:
    """Return a tuple of all the integers in text."""
    return tuple(map(int, re.findall('-?[0-9]+', text)))

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    """Parse text into atoms (numbers or strs), possibly ignoring a regex."""
    if ignore:
        text = re.sub(ignore, '', text)
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Union[float, int, str]:
    """Parse text into a single float or int or str."""
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text
    
def dotproduct(A, B) -> float: 
    return sum(a * b for a, b in zip(A, B))

def mapt(fn, *args):
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

cat = ''.join
flatten = chain.from_iterable
Char = str # Type used to indicate a single character

# Day 1: Sonar Sweep

In [4]:
in1: List[int] = data(1, int)

In [5]:
def day1_1(nums: List[int]) -> int:
    return sum([1 for i in range(len(nums) - 1) if nums[i+1] > nums[i]])

def day1_2(nums: List[int]) -> int:
    return day1_1([sum(nums[i:i+3]) for i in range(len(nums) - 2)])

In [6]:
do(1, 1521, 1543)

[1521, 1543]

# Day 2: Dive!

In [7]:
Command = Tuple[str, int]

def parse_command(line: str) -> Command:
    direction, quantity = line.split(' ')
    return (direction, int(quantity))

in2: List[Command] = data(2, parse_command)


In [8]:
def day2_1(commands: List[Command]) -> int:
    pos = [0, 0]
    for command in commands:
        direction = command[0]
        quantity = command[1]
        if direction == "forward":
            pos[0] += quantity
        if direction == "up":
            pos[1] -= quantity
        if direction == "down":
            pos[1] += quantity
    return math.prod(pos)

def day2_2(commands: List[Command]) -> int:
    pos = [0, 0]
    aim = 0
    for command in commands:
        direction = command[0]
        quantity = command[1]
        if direction == "forward":
            pos[0] += quantity
            pos[1] += quantity * aim
        if direction == "up":
            aim -= quantity
        if direction == "down":
            aim += quantity
    return math.prod(pos)

In [9]:
do(2, 1383564, 1488311643)

[1383564, 1488311643]

# Day 3: Binary Diagnostic

In [10]:
BinaryReportLine = List[int]

def parse_binary_report_line(line: str) -> BinaryReportLine:
    return list(map(int, list(line)))

in3: List[BinaryReportLine] = data(3, parse_binary_report_line)

In [11]:
def day3_1(report_lines: List[BinaryReportLine]) -> int:
    inverted = list(map(list, zip(*report_lines)))
    gamma = "".join(["1" if sum(col) > len(col)/2 else "0" for col in inverted])
    epsilon = "".join(["1" if v == "0" else "0" for v in gamma])
    return int(gamma, 2) * int(epsilon, 2)

def filter_report(report_lines: List[BinaryReportLine], filter_value_fn) -> BinaryReportLine:
    filtered = report_lines
    i = 0
    while len(filtered) > 1:
        column_vals = [row[i] for row in filtered]
        total = len(column_vals)
        ones = sum(column_vals)
        filter_value = filter_value_fn(total, ones)
        filtered = [row for row in filtered if row[i] == filter_value]
        i+=1
    return filtered[0]

def day3_2(report_lines: List[BinaryReportLine]) -> int:
    oxygen = filter_report(report_lines, lambda total, ones: 1 if ones >= total/2 else 0)
    co2 = filter_report(report_lines, lambda total, ones: 1 if ones < total/2 else 0)
    return int("".join(map(str, oxygen)), 2) * int("".join(map(str, co2)), 2)



In [12]:
do(3, 2498354, 3277956)

[2498354, 3277956]

# Day 4: Giant Squid

In [13]:
def day4_data() -> Tuple[List[str], List[List[List[str]]]]:
    parsed_data = data(4, sep="\n\n")
    called_numbers = parsed_data[0].split(",")
    boards = []
    for unparsed_board in parsed_data[1:]:
        board = []
        board_lines = unparsed_board.split("\n")
        for line in board_lines:
            board.append(line.split())
        boards.append(board)
    return called_numbers, boards

in4: Tuple[List[str], List[List[List[str]]]] = day4_data()

In [14]:
in4

class Board:
    def __init__(self, board):
        self.board = board
        self.wins = []
        for row in board:
            self.wins.append(set(row))

        self.won = False

        for i in range(len(board[0])):
            col_win = set()
            for row in board:
                col_win.add(row[i])
            self.wins.append(col_win)

    def all_values(self) -> List[str]:
        return list(set([item for row in self.board for item in row]))

    def get_score(self) -> int:
        uncalled_values = set()
        for win in self.wins:
            uncalled_values |= win
        return sum(map(int, uncalled_values))


    
# Return a map from the numbers in the boards to which board they are in
# For eact board, build a set of all the possible winning paths
def build_bingo_map(boards: List[List[List[str]]]) -> Dict[str, List[Board]]:
    m = defaultdict(list)
    for board in boards:
        b = Board(board)
        for v in b.all_values():
            m[v].append(b)
    return m


def day4_1(data: Tuple[List[str], List[List[List[str]]]]) -> int:
    calls, boards = data
    bingo_map = build_bingo_map(boards)
    for call in calls:
        boards_with_number = bingo_map[call]
        for board in boards_with_number:
            won = False
            for win in board.wins:
                if call in win:
                    win.remove(call)
                    if len(win) == 0:
                        won = True
            if won:
                return board.get_score() * int(call)

    return 0

def day4_2(data: Tuple[List[str], List[List[List[str]]]]) -> int:
    winning_boards = []
    calls, boards = data
    bingo_map = build_bingo_map(boards)
    for call in calls:
        boards_with_number = bingo_map[call]
        for board in boards_with_number:
            won = False
            for win in board.wins:
                if call in win:
                    win.remove(call)
                    if len(win) == 0:
                        won = True
            if won and not board.won:
                winning_boards.append((board, int(call)))
                board.won = True
        if len(winning_boards) == len(boards):
            return winning_boards[-1][0].get_score() * winning_boards[-1][1]

    return 0


do(4, 41668, 10478)    


[41668, 10478]

# Day 5: Hydrothermal Venture

In [15]:
VentLine = Tuple[Tuple[int, int], Tuple[int, int]]

def parse_day5(line: str) -> VentLine:
    return tuple(mapt(int, v.strip().split(",")) for v in line.split("->"))

in5: List[VentLine] = data(5, parse_day5)

In [16]:
def day5_1(vent_lines: List[VentLine]) -> int:
    vents = defaultdict(int)
    for line in vent_lines:
        if line[0][0] == line[1][0]:
            a = line[0][1]
            b = line[1][1]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(line[0][0], i)] += 1
        if line[0][1] == line[1][1]:
            a = line[0][0]
            b = line[1][0]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(i, line[0][1])] += 1
        
    return sum([1 for count in vents.values() if count >= 2])

def get_step(start: int, end: int) -> int:
    if end > start:
        return 1
    if start > end:
        return -1
    return 0

def day5_2(vent_lines: List[VentLine]) -> int:
    vents = defaultdict(int)
    for line in vent_lines:
        if line[0][0] == line[1][0]:
            a = line[0][1]
            b = line[1][1]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(line[0][0], i)] += 1
        elif line[0][1] == line[1][1]:
            a = line[0][0]
            b = line[1][0]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(i, line[0][1])] += 1
        else:
            # Diagonal
            xstep = get_step(line[0][0], line[1][0])
            ystep = get_step(line[0][1], line[1][1])
            for (x, y) in zip(range(line[0][0], line[1][0] + xstep, xstep), range(line[0][1], line[1][1] + ystep, ystep)):
                vents[(x, y)] += 1
        
    return sum([1 for count in vents.values() if count >= 2])

do(5, 4728, 17717)


[4728, 17717]

# Day 6: Lanternfish

In [17]:
def parse_day5(line: str) -> VentLine:
    return tuple(mapt(int, v.strip().split(",")) for v in line.split("->"))

in6: List[int] = first(data(6, lambda l: list(map(int, l.split(",")))))

In [18]:
def lanternfish_after_n_days(timers: List[int], days: int) -> int:
    m = defaultdict(int)
    for timer in timers:
        m[timer] += 1

    for i in range(days):
        next_day = defaultdict(int)
        for t in m:
            if t == 0:
                next_day[8] += m[t]
                next_day[6] += m[t]
            else:
                next_day[t-1] += m[t]
        m = next_day
    return sum(count for count in m.values())

def day6_1(timers: List[int]) -> int:
    return lanternfish_after_n_days(timers, 80)

def day6_2(timers: List[int]) -> int:
    return lanternfish_after_n_days(timers, 256)

do(6, 386640, 1733403626279)

[386640, 1733403626279]